## 개요

LangGraph는 LLM을 사용하여 상태 기반의 다중 행위자 애플리케이션을 구축하기 위한 라이브러리로, 에이전트 및 다중 에이전트 워크플로우를 만드는 데 사용됩니다. 다른 LLM 프레임워크와 비교할 때 다음과 같은 핵심 장점을 제공합니다: 사이클, 제어 가능성, 지속성. LangGraph는 대부분의 에이전트 아키텍처에 필수적인 사이클을 포함하는 흐름을 정의할 수 있게 해주며, 이는 DAG 기반 솔루션과 차별화되는 점입니다. 매우 저수준의 프레임워크로서 애플리케이션의 흐름과 상태에 대한 세밀한 제어를 제공하여 신뢰할 수 있는 에이전트를 만드는 데 중요합니다. 또한, LangGraph에는 고급 human-in-the-loop 및 메모리 기능을 가능하게 하는 내장 지속성이 포함되어 있습니다.

LangGraph는 LangChain의 창립자인 LangChain Inc에서 개발했지만 LangChain 없이도 사용할 수 있습니다.


## 주요 기능

- **사이클 및 분기**: 애플리케이션에서 루프와 조건문을 구현할 수 있습니다.
- **지속성**: 그래프의 각 단계 후 자동으로 상태를 저장합니다. 그래프 실행을 언제든지 일시 중지하고 다시 시작하여 오류 복구, human-in-the-loop 워크플로우, 시간 여행 등을 지원합니다.
- **Human-in-the-Loop**: 에이전트가 계획한 다음 작업을 승인하거나 수정하기 위해 그래프 실행을 중단할 수 있습니다.
- **스트리밍 지원**: 각 노드에서 생성된 출력(토큰 스트리밍 포함)을 실시간으로 스트리밍할 수 있습니다.
- **LangChain과의 통합**: LangGraph는 LangChain 및 LangSmith와 원활하게 통합됩니다(필수는 아님).


### Installation

In [2]:
!pip install -U langgraph

### gpt사용을 위한 openAi API key 셋팅

In [3]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# 환경 변수에서 OpenAI API 키를 가져옵니다.
openai_api_key = os.getenv("OPENAI_API_KEY")

# API 키가 존재하지 않는 경우 에러를 발생시킵니다.
if not openai_api_key:
    # API 키가 없다는 에러 메시지를 출력하고 프로그램을 종료합니다.
    raise ValueError("OpenAI API 키가 없습니다. 한 번 더 확인 부탁드립니다.")

In [4]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode


# 에이전트가 사용할 도구 정의
@tool
def search(query: str):
    """웹을 탐색하기 위한 호출."""
    # 이것은 자리 표시자입니다. LLM에게는 비밀로...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "60도에 안개가 낀 날씨입니다."
    return "90도에 맑은 날씨입니다."


tools = [search]

tool_node = ToolNode(tools)

# 모델을 GPT로 변경
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools(tools)

# 계속할지 여부를 결정하는 함수 정의
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # LLM이 도구 호출을 하면 "tools" 노드로 라우팅
    if last_message.tool_calls:
        return "tools"
    # 그렇지 않으면 중지(사용자에게 응답)
    return END


# 모델을 호출하는 함수 정의
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # 기존 목록에 추가되기 때문에 목록을 반환합니다.
    return {"messages": [response]}


# 새로운 그래프 정의
workflow = StateGraph(MessagesState)

# 사이클링할 두 노드 정의
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 진입점을 `agent`로 설정
# 이는 첫 번째로 호출되는 노드를 의미합니다.
workflow.set_entry_point("agent")

# 조건부 경로 추가
workflow.add_conditional_edges(
    # 먼저, 시작 노드를 정의합니다. `agent`를 사용합니다.
    # 이는 `agent` 노드가 호출된 후의 경로를 의미합니다.
    "agent",
    # 다음으로, 어느 노드가 다음에 호출될지 결정하는 함수를 전달합니다.
    should_continue,
)

# `tools`에서 `agent`로의 일반 경로 추가
# 이는 `tools`가 호출된 후 `agent` 노드가 다음에 호출된다는 것을 의미합니다.
workflow.add_edge("tools", 'agent')

# 그래프 실행 간 상태를 유지하기 위해 메모리 초기화
checkpointer = MemorySaver()

# 마지막으로 컴파일합니다!
# 이를 LangChain Runnable로 컴파일하여,
# 다른 runnable처럼 사용할 수 있게 합니다.
# 그래프를 컴파일할 때 메모리를 (옵션으로) 전달하는 것을 주의하세요.
app = workflow.compile(checkpointer=checkpointer)

# Runnable 사용
final_state = app.invoke(
    {"messages": [HumanMessage(content="sf 날씨가 어떤가요?")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'현재 샌프란시스코의 날씨는 60도이며 안개가 낀 상태입니다.'

이제 동일한 "thread_id"를 전달하면, 저장된 상태(즉, 저장된 메시지 목록)를 통해 대화 문맥이 유지됩니다.

In [5]:
# 동일한 "thread_id"를 사용하여 Runnable을 다시 호출
final_state = app.invoke(
    {"messages": [HumanMessage(content="내일 날씨는 어떤가요?")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'내일 샌프란시스코의 날씨는 60도이며 안개가 낀 상태입니다.'

# LangGraph 단계별 설명

## 1. 모델 및 도구 초기화

- LLM으로 `ChatAnthropic`을 사용합니다. **주의**: 모델이 이러한 도구를 호출할 수 있도록 모델에 이를 알려야 합니다. 이는 LangChain 도구를 OpenAI 도구 호출 형식으로 변환하여 `.bind_tools()` 메서드를 사용해 수행할 수 있습니다.
- 사용하려는 도구를 정의합니다 - 여기서는 검색 도구를 사용합니다. 자체 도구를 만드는 것은 매우 쉽습니다 - 이를 수행하는 방법은 [여기](#) 문서를 참조하십시오.

## 2. 상태를 사용하여 그래프 초기화

- 상태 스키마(여기서는 `MessagesState`)를 전달하여 그래프(`StateGraph`)를 초기화합니다.
- `MessagesState`는 LangChain `Message` 객체 목록과 각 노드에서 상태로 업데이트를 병합하는 로직을 포함하는 미리 정의된 상태 스키마입니다.

## 3. 그래프 노드 정의

- 필요한 주요 노드 두 개가 있습니다:
  - `agent` 노드: 어떤 동작(있다면)을 취할지 결정하는 책임을 가집니다.
  - `tools` 노드: 에이전트가 동작을 취하기로 결정하면 이 노드가 해당 동작을 실행합니다.

## 4. 진입점 및 그래프 경로 정의

- 먼저, 그래프 실행을 위한 진입점을 설정해야 합니다 - `agent` 노드.
- 그런 다음 하나의 일반 경로와 하나의 조건부 경로를 정의합니다. 조건부 경로는 그래프 상태의 내용(즉, `MessagesState`)에 따라 목적지가 결정된다는 것을 의미합니다. 우리의 경우, 목적지는 `agent`(LLM)가 결정할 때까지 알 수 없습니다.
  - 조건부 경로: `agent`가 호출된 후, 다음을 수행해야 합니다:
    a. 에이전트가 동작을 취하라고 하면 도구를 실행하거나,
    b. 에이전트가 도구 실행을 요청하지 않았다면 종료(사용자에게 응답).
  - 일반 경로: 도구가 호출된 후, 그래프는 항상 다음에 어떤 작업을 할지 결정하기 위해 `agent`로 돌아와야 합니다.

## 5. 그래프 컴파일

- 그래프를 컴파일하면, 이를 LangChain `Runnable`로 변환하여 자동으로 `.invoke()`, `.stream()` 및 `.batch()`를 입력과 함께 호출할 수 있습니다.
- 또한 체크포인터 객체를 전달하여 그래프 실행 간 상태를 지속시키고, 메모리, human-in-the-loop 워크플로우, 시간 여행 등을 활성화할 수 있습니다. 우리의 경우 간단한 메모리 체크포인터인 `MemorySaver`를 사용합니다.

## 6. 그래프 실행

a. LangGraph는 입력 메시지를 내부 상태에 추가한 다음 상태를 진입점 노드인 `agent`에 전달합니다.
b. `agent` 노드가 실행되어 채팅 모델을 호출합니다.
c. 채팅 모델이 `AIMessage`를 반환합니다. LangGraph는 이를 상태에 추가합니다.
d. 그래프는 `AIMessage`에 더 이상 `tool_calls`가 없을 때까지 다음 단계를 반복합니다:
  - `AIMessage`에 `tool_calls`가 있으면, "tools" 노드가 실행됩니다.
  - `agent` 노드가 다시 실행되어 `AIMessage`를 반환합니다.
e. 실행이 특별한 `END` 값으로 진행되고 최종 상태를 출력합니다. 그 결과, 모든 채팅 메시지 목록을 출력으로 얻습니다.

이 모든 단계를 통해 LangGraph를 사용하여 상태 기반의 다중 행위자 애플리케이션을 구축할 수 있습니다. 이를 통해 에이전트와 다중 에이전트 워크플로우를 쉽게 관리하고 확장할 수 있습니다.
